# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [39]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
from ConsumerProblem import HouseholdSpecializationModelClass


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [75]:
# code for solving the model (remember documentation and comments)
#Household = HouseholdSpecializationModelClass()

alpha_list = [0.25, 0.50, 0.75]
sigma_list = [0.5, 1.0, 1.5]

Household = HouseholdSpecializationModelClass()

res_list = []
#c. main
for i_alpha,alpha in enumerate(alpha_list):
    for i_sigma,sigma in enumerate(sigma_list):
            Household.par.alpha = alpha
            Household.par.sigma = sigma
            result = Household.solve_discrete()
            #res_list.append(result.HF/result.HM)
            print(result.HF/result.HM,alpha,sigma)
            #print(result.HF/result.HM)

print(res_list)

0.3076923076923077 0.25 0.5
1.0 0.5 0.5
3.25 0.75 0.5
0.3076923076923077 0.25 1.0
1.0 0.5 1.0
3.25 0.75 1.0
0.3076923076923077 0.25 1.5
1.0 0.5 1.5
3.25 0.75 1.5
[]


In [43]:
alpha = .5
sigma = .25
Household.par.alpha = alpha
Household.par.alpha = sigma
result = Household.solve_discrete()
HF_HM= result.HF/result.HM


In [51]:
HF_HM= result.HF/result.HM
f


0.3076923076923077


# Question 2

Explain your code and procedure

In [23]:
# code

# Question 3

Explain your code and procedure

In [24]:
# code

ADD CONCISE CONLUSION.